# LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, `Runnable` 프로토콜을 구현했다.

`Runnable` 프로토콜은 대부분 컴포넌트에 구현되어 있다.

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [10]:
model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=512
)

template = PromptTemplate.from_template("{tech}에 대하여 3문장으로 설명해줘")

chain = template | model | StrOutputParser()

## stream : 실시간 출력
이 함수는 스트림을 반복하여 각 데이터의 내용(context)을 즉시 출력한다.

end=""인자는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True 인자는 출력 버퍼를 즉시 비우도록 한다.

In [11]:
for token in chain.stream("비행기"):
    print(token,end="",flush=True)

비행기는 공중에서 사람이나 화물을 운송하기 위해 설계된 항공기입니다. 일반적으로 날개와 엔진을 갖추고 있으며, 공기역학 원리를 이용해 비행합니다. 현대의 비행기는 상업용 여객기부터 군사기, 화물기 등 다양한 종류가 있습니다.

## invoke : 호출
invoke 메서드는 주제를 인자로 받아 해당 주자에 대한 처리를 수행한다.

In [12]:
chain.invoke("비행기")

'비행기는 공중을 비행할 수 있도록 설계된 교통 수단으로, 주로 사람이나 화물을 운송하는 데 사용됩니다. 항공기의 비행 원리는 양력, 중력, 추진력, 항력의 상호 작용에 기반하며, 이를 통해 하늘을 날 수 있습니다. 현대의 비행기는 다양한 기술과 안전 장치를 갖추고 있어 효율적이고 안전한 항공 여행을 제공합니다.'

# batch : 배치(단위 실행)
- 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 디셔너리에 있는 topic 키의 값을 사용하여 일괄 처리

In [14]:
answer = chain.batch([{"tech":"컴퓨터"},{"tech":"머그컵"}])

In [15]:
answer[0]

'컴퓨터는 데이터를 처리하고 저장하는 전자 기기로, 다양한 작업을 수행할 수 있는 프로그램을 실행할 수 있습니다. 일반적으로 하드웨어와 소프트웨어로 구성되어 있으며, 하드웨어는 물리적인 부품, 소프트웨어는 이들 부품을 제어하는 프로그램을 포함합니다. 컴퓨터는 정보 검색, 계산, 통신 등 여러 분야에서 중요한 역할을 하고 있습니다.'

In [16]:
answer[1]

'머그컵은 일반적으로 손잡이가 있는 깊고 넓은 컵으로, 주로 뜨거운 음료를 마시는 데 사용됩니다. 다양한 디자인과 재질로 제작되어 개인의 취향에 맞게 선택할 수 있으며, 커피, 차, 핫초코 등 다양한 음료를 담기에 적합합니다. 또한, 머그컵은 종종 선물용으로도 인기가 있으며, 개인화된 메시지나 이미지로 장식되기도 합니다.'

max_concurrency 매개변수를 사용하여 동시 요청 수를 설정할 수 있다.

config 딕셔너리는 max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정한다.

In [18]:
answer = chain.batch(
    [
        {"tech": "ChatGPT"},
        {"tech": "Instagram"},
        {"tech": "멀티모달"},
        {"tech": "프로그래밍"},
        {"tech": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

In [19]:
answer

['ChatGPT는 OpenAI에서 개발한 인공지능 언어 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 자연스럽고 유창한 텍스트로 응답할 수 있도록 훈련되었습니다. 다양한 주제에 대해 대화할 수 있으며, 정보 제공, 문제 해결, 창의적인 글쓰기 등의 용도로 활용됩니다.',
 'Instagram은 사용자가 사진과 동영상을 공유하고 소통할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자들이 창의적으로 콘텐츠를 제작할 수 있도록 돕습니다. 또한, 스토리, IGTV, 리얼스와 같은 기능을 통해 실시간으로 소통하고 다양한 형식의 콘텐츠를 소비할 수 있는 환경을 제공합니다.',
 '멀티모달(Multimodal)이라는 개념은 다양한 형태의 데이터 또는 정보(예: 텍스트, 이미지, 오디오 등)를 결합하여 처리하거나 분석하는 방식을 의미합니다. 이러한 접근은 서로 다른 모드 간의 상호작용을 통해 더 풍부하고 정확한 이해를 가능하게 합니다. 최근 인공지능 분야에서는 멀티모달 학습이 자연어 처리와 컴퓨터 비전 기술을 통합하여 더욱 효과적인 모델을 개발하는 데 활용되고 있습니다.',
 '프로그래밍은 컴퓨터가 특정 작업을 수행하도록 지시하는 과정으로, 주로 프로그래밍 언어를 사용하여 코드로 작성됩니다. 이 과정에서는 알고리즘을 설계하고, 문제를 해결하기 위한 로직을 구현하며, 결과를 검증하는 단계를 포함합니다. 프로그래밍은 소프트웨어 개발, 데이터 분석, 웹 개발 등 다양한 분야에서 활용되며, 현대 사회에서 필수적인 기술로 자리 잡고 있습니다.',
 '머신러닝은 데이터에서 패턴을 학습하고 예측을 수행하는 인공지능의 한 분야입니다. 알고리즘을 사용하여 주어진 데이터를 분석하고, 이를 통해 새로운 데이터에 대한 결정을 자동으로 내릴 수 있도록 합니다. 머신러닝은 이미지 인식, 자연어 처리, 추천 시스템 등 다양한 분야에서 활용되고 있습니다.']

# async stream : 비동기 스트림

chain.astream 은 비동기 스트림을 생성하며, 주어진 토픽에 대한 정보를 비동기적으로 처리한다.

비동기 for루프(async for)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, print 함수를 통해 메시지의 내용(content)를 즉시 출력한다.

In [20]:
async for token in chain.astream({"tech":"나무망치"}):
    print(token,end="",flush=True)

나무망치는 주로 나무로 만들어진 도구로, 주로 못을 박거나 물체를 부드럽게 두드리는 데 사용됩니다. 금속망치에 비해 충격이 덜해 손상 위험이 낮아 섬세한 작업에 적합합니다. 또한, 나무망치는 가벼운 무게 덕분에 사용하기 쉽고 다양한 작업에 활용될 수 있습니다.

# async invoke : 비동기 호출

chain 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행한다.

In [21]:
async_process = chain.ainvoke({"tech":"맥북"})

In [22]:
await async_process

'맥북은 애플이 개발한 노트북 컴퓨터로, macOS 운영 체제를 사용합니다. 고급스러운 디자인과 뛰어난 성능을 갖추고 있으며, 사용자 친화적인 인터페이스로 많은 사람들에게 사랑받고 있습니다. 또한, 애플의 생태계와 잘 통합되어 있어 아이폰, 아이패드 등과의 연동이 용이합니다.'

# async batch : 비동기 배치

`abatch` 비동기적으로 일련의 작업을 일괄 처리

In [23]:
async_batch = chain.abatch(
    [
        {"tech": "ChatGPT"},
        {"tech": "Instagram"},
    ],
)

In [24]:
await async_batch

['ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 자연어 처리를 통해 사용자와의 대화를 지원합니다. 이 모델은 방대한 양의 텍스트 데이터를 학습하여 다양한 주제에 대해 질문에 답하거나 정보를 제공할 수 있습니다. 사용자는 ChatGPT를 통해 정보 검색, 글쓰기 보조, 그리고 다양한 주제에 대한 토론 등을 할 수 있습니다.',
 '인스타그램은 사용자들이 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자가 창의적으로 콘텐츠를 꾸밀 수 있도록 돕습니다. 또한, 친구들과의 소통뿐만 아니라 브랜드와의 연결을 통해 마케팅 및 비즈니스 기회도 제공합니다.']